# I. Prepare data

In [1]:
from tqdm import tqdm
from lxml import etree
import os, nltk

In [2]:
topic_tree = etree.parse('data/topics2016.xml')

def get_topic(i):# returns the summary string of the ith topic
    summary = topic_tree.xpath('//topic[@number="%d"]/summary/text()'%i)[0]
    return str(summary).lower()

# build a mapping of article name (PMCID) to its file path

PMC_PATH = '/local/XW/DATA/TREC/PMCs/'
pmcid2fpath = {}

for subdir1 in os.listdir(PMC_PATH):
    for subdir2 in os.listdir(os.path.join(PMC_PATH, subdir1)):
        diry = os.path.join(PMC_PATH, subdir1, subdir2)
#         print diry, len(os.listdir(diry))
        for fn in os.listdir(diry):
            pmcid = fn[:-5]
            fpath = os.path.join(diry, fn)
            pmcid2fpath[pmcid] = fpath

def get_article_abstract(pmcid):
    fpath = pmcid2fpath[pmcid]
    tree = etree.parse(fpath)
    abstract = tree.xpath('//abstract')[0]
    ret = u''+abstract.xpath('string(.)')
    if len(ret.split())<20: raise Exception('abstraction too short:'+pmcid)
    return ret.lower()

In [3]:
corpus = []
pmcid_2relevance = [{} for i in xrange(31)] # list of dict mapping pmcid to relevance
with open('data/qrels.txt') as f:
    for line in tqdm(f, total=37707): 
        topicid, _, pmcid, relevance = line.split()
        topicid = int(topicid)
        try:
            corpus.append(get_article_abstract(pmcid)) # !some articles don't have an abstract!
            pmcid_2relevance[topicid][pmcid] = int(relevance)
        except: pass

100%|██████████| 37707/37707 [05:47<00:00, 108.38it/s]


In [4]:
print '%d articles are retrieved' % len(corpus)

34289 articles are retrieved


In [199]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(corpus)

def get_idf(wd):
    if wd not in vectorizer.vocabulary_: return -1e10
    return vectorizer.idf_[ vectorizer.vocabulary_[wd] ]

vocab = set(vectorizer.vocabulary_.keys())

In [6]:
import os, sys, time
import numpy as np
from numpy.linalg import norm
import pandas as pd
from tqdm import tqdm
import cPickle as pk
np.random.seed(1) # to be reproducible

In [7]:
W2V_FPATH = '/local/XW/DATA/WORD_EMBEDDINGS/biomed-w2v-200.txt'
GLOVE_FPATH = '/local/XW/DATA/WORD_EMBEDDINGS/glove.6B.200d.txt'

In [8]:
word2vec = {} # maps word ---> embedding vector
with open(W2V_FPATH) as f:
    for line in tqdm(f, total=5443657): #5443657 400000
        vals = line.split()
        word = vals[0]
        if word in vocab:
            vec = np.asarray(vals[1:], dtype='float')
            word2vec[word] = vec
print 'found %d word vectors.' % len(word2vec)

100%|██████████| 5443657/5443657 [01:00<00:00, 89801.52it/s]

found 54911 word vectors.


In [9]:
from nltk.corpus import stopwords
stopwds = set(stopwords.words('english'))

In [10]:
get_topic(17)

'76-year-old female with personal history of diastolic congestive heart failure, atrial fibrillation on coumadin, presenting with low hematocrit and dyspnea.'

In [11]:
_queries = [get_topic(i) for i in xrange(1,21)] # top 20 topics' queries
QUERIES = []
for q in _queries:
    q2 = [wd for wd in q.split() if (wd not in stopwds) and (wd in word2vec)] # filter out stopword and words not in w2v
    QUERIES.append(q2)

In [12]:
map(len, QUERIES)

[7, 14, 26, 11, 12, 20, 16, 11, 13, 12, 14, 13, 23, 11, 40, 16, 11, 19, 20, 16]

In [43]:
N = 40 # = max query length

In [17]:
WD_PLACEHOLDER = '</s>'
def pad_query(q, SZ=40):
    return q + [WD_PLACEHOLDER]*(SZ-len(q))
QUERIES = map(pad_query, QUERIES)

## helper functions

In [20]:
randvec = np.random.randn(200)
def get_histvec(q_wd, doc):
    if q_wd == WD_PLACEHOLDER: return np.zeros(30)
    qvec = word2vec.get(q_wd, randvec)
#     dvecs = np.vstack( [word2vec.get(wd, randvec) for wd in nltk.word_tokenize(doc)] )
    words_doc = filter(lambda wd:wd in word2vec, nltk.word_tokenize(doc))
    dvecs = np.vstack( [ word2vec[wd] for wd in words_doc ] )
    cossims = np.dot(dvecs, qvec) / norm(qvec) / norm(dvecs, axis=1)
    hist, _ = np.histogram( cossims[cossims<1.0], bins=29, range=(-1,1) )
    ones = len(cossims) - sum(hist)
    ret = np.array( list(hist) + [ones] )
    return ret # np.reshape(ret, (-1, 30))

In [21]:
def get_query_doc_feature(query, pmcid): # query: list of words
    doc = get_article_abstract(pmcid)
    return np.array([ get_histvec(wd, doc) for wd in query])

### prepare data

In [184]:
pos_ids, neg_ids = [], [] # pos_ids[q] is a list (positive pmcids for query `q`)
hists_pos, hists_neg = [], [] # hists_pos[q] is a list (positive hists for query `q`)
                              # hists_pos[q][i] is an array of size N*30 (the ith hists-feature array for query q)
for topic in xrange(1,6):
    query = QUERIES[topic-1]
    pos_ids_q, neg_ids_q = [], []
    hists_pos_q, hists_neg_q = [], []
    relevance = pmcid_2relevance[topic]
    for pmcid in tqdm(relevance.keys()):
        if relevance[pmcid]==0: 
            neg_ids_q.append(pmcid)
            hists_neg_q.append(get_query_doc_feature(query,pmcid))
        else: 
            pos_ids_q.append(pmcid)
            hists_pos_q.append(get_query_doc_feature(query,pmcid))
    hists_pos_q, hists_neg_q = map(np.array, [hists_pos_q, hists_neg_q])
    hists_pos.append(hists_pos_q); hists_neg.append(hists_neg_q)
    pos_ids.append(pos_ids_q); neg_ids.append(neg_ids_q)
print len(pos_ids), len(neg_ids)

100%|██████████| 1372/1372 [00:59<00:00, 22.99it/s]

5 5


### prepare generator

In [183]:
VALDATION_SPLIT = 0.2
BATCH_SZ = 128
NB_EPOCH = 20

In [185]:
print map(len, hists_pos)
print map(len, hists_neg)

[124, 34, 140, 18, 94]
[1204, 1153, 1196, 1300, 1278]


In [188]:
idx_pairs = [] # list of tuples of form (q, pos_idx, neg_idx)
for q in xrange(5):
    hists_pos_q, hists_neg_q = hists_pos[q], hists_neg[q]
    for pidx in xrange(len(hists_pos_q)):
        for nidx in xrange(len(hists_neg_q)):
            idx_pairs.append( (q, pidx, nidx) )
idx_pairs = np.array(idx_pairs)
np.random.shuffle(idx_pairs)
val_sz = int(len(idx_pairs)*VALDATION_SPLIT)
idx_pairs_train, idx_pairs_val = idx_pairs[val_sz:], idx_pairs[:val_sz]

print idx_pairs_train.shape, idx_pairs_val.shape

(399576, 3) (99894, 3)


In [189]:
idx_batch = idx_pairs_train[:10]
zip(*idx_batch)

[(2, 0, 0, 4, 2, 0, 0, 2, 2, 0),
 (6, 74, 80, 20, 40, 55, 122, 83, 26, 43),
 (1132, 812, 835, 493, 1167, 1115, 1124, 384, 701, 275)]

In [200]:
IDFs = [ np.array ([ get_idf(wd) for wd in query]) for query in QUERIES]
IDFs = np.array(IDFs)

In [202]:
def batch_generator(idx_pairs, batch_size=BATCH_SZ): 
    np.random.shuffle(idx_pairs)
    batches_pre_epoch = len(idx_pairs) // batch_size
    samples_per_epoch = batches_pre_epoch * batch_size # make samples_per_epoch a multiple of batch size
    counter = 0
    y_true_batch_dummy = np.ones((batch_size))
    while 1:
        idx_batch = idx_pairs[batch_size*counter: min(samples_per_epoch, batch_size*(counter+1))]
#         q_batch, pidx_batch, nidx_batch = zip(*idx_batch)
#         idfs_batch = IDFs[q_batch,]
#         pos_batch = hists_pos[q_batch, pidx_batch]
#         neg_batch = hists_neg[q_batch, nidx_batch]
        idfs_batch, pos_batch, neg_batch = [], [], []
        for q, pidx, nidx in idx_batch:
            idfs_batch.append(IDFs[q])
            pos_batch.append(hists_pos[q][pidx])
            neg_batch.append(hists_neg[q][nidx])
        idfs_batch, pos_batch, neg_batch = map(np.array, [idfs_batch, pos_batch, neg_batch])
#         print idfs_batch.shape, pos_batch.shape, neg_batch.shape
        counter += 1
        if (counter >= batches_pre_epoch):
            np.random.shuffle(idx_pairs)
            counter=0
        
        yield [idfs_batch, pos_batch, neg_batch], y_true_batch_dummy

# II. Define the deep relevance model

In [40]:
# define a function for visualization of model
import pydot
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot
def viz_model(model):
    return SVG(model_to_dot(model).create(prog='dot', format='svg'))

Using TensorFlow backend.


### construct the relevance IR model

In [193]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, InputLayer, Flatten, Input, Merge, merge, Reshape
import keras.backend as K
from keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf

In [194]:
# 2 main components of the structure: feed forward network and gating
feed_forward = Sequential(
    [Dense(input_dim=30, output_dim=5, activation='tanh'),
     Dense(output_dim=1, activation='tanh')], 
    name='feed_forward_nw')

# ***note: have to wrap ops into Lambda layers !!***
# cf: https://groups.google.com/forum/#!topic/keras-users/fbRS-FkZw_Q
from keras.layers.core import Lambda

input_idf = Input(shape=(N,), name='input_idf')
def scale(x): 
    w = K.variable(1, name='w_g')
    return K.mul(x,w)
def scale_output_shape(input_shape): return input_shape

scaled = Lambda(scale, scale_output_shape, name='softmax_scale')(input_idf)
gs_out = Activation('softmax', name='softmax')(scaled)
gating = Model(input=input_idf, output=gs_out, name='gating')

# first input: hist vectors
input_hists = Input(shape=(N,30), name='input_hists')

def slicei(x, i): return x[:,i,:]
def slicei_output_shape(input_shape): return (input_shape[0], input_shape[2])
zs = [ feed_forward( Lambda(lambda x:slicei(x,i), slicei_output_shape, name='slice%d'%i)(input_hists) )\
          for i in xrange(N) ]

def concat(x): return K.concatenate(x) 
def concat_output_shape(input_shape): return (input_shape[0][0], N)
zs = Lambda(concat, concat_output_shape, name='concat_zs')(zs)

# second input: idf scores of each query term 
input_idf = Input(shape=(N,), name='input_idf')
gs = gating(input_idf)

def innerprod(x): return K.sum( K.mul(x[0],x[1]), axis=1)
def innerprod_output_shape(input_shape): return (input_shape[0][0],1)
scores = Lambda(innerprod, innerprod_output_shape, name='innerprod_zs_gs')([zs, gs])

scoring_model = Model(input=[input_idf, input_hists], output=[scores], name='scoring_model')

# third input -- the negative hists vector 
input_hists_neg = Input(shape=(N,30), name='input_hists_neg')

zs_neg = [ feed_forward( Lambda(lambda x:slicei(x,i), slicei_output_shape, name='slice%d_neg'%i)(input_hists_neg) )\
          for i in xrange(N) ]

zs_neg = Lambda(concat, concat_output_shape, name='concat_zs_neg')(zs_neg)

scores_neg = Lambda(innerprod, innerprod_output_shape, name='innerprod_zs_gs_neg')([zs_neg, gs])

two_score_model = Model(input=[input_idf, input_hists, input_hists_neg], 
                        output=[scores, scores_neg], 
                        name='two_score_model')

def diff(x): return tf.sub(x[0], x[1]) #x[0]-x[1]
def diff_output_shape(input_shape): return input_shape[0]
posneg_score_diff = Lambda(diff, diff_output_shape, name='posneg_score_diff')([scores, scores_neg])
ranking_model = Model(input=[input_idf, input_hists,  input_hists_neg]
                      , output=[posneg_score_diff]
                      , name='ranking_model')

# define my loss function: hinge of score_pos - score_neg
def pairwise_hinge(y_true, y_pred): # y_pred = score_pos - score_neg, **y_true doesn't matter here**
    return K.mean( K.maximum(1. - y_pred, y_true*0.0) )  

ranking_model.compile(optimizer='adagrad', loss=pairwise_hinge)

# III. train model (for topic-1)

## train model using `fit_generator`

In [195]:
# self-defined metrics
def ranking_acc(y_true, y_pred):
    y_pred = y_pred > 0 
    return K.mean(y_pred)

In [196]:
ranking_model.compile(optimizer='adagrad', loss=pairwise_hinge, metrics=[ranking_acc])

In [203]:
logdir = './log/relevance_matching'
_callbacks = [ EarlyStopping(monitor='val_loss', patience=2),
               TensorBoard(log_dir=logdir, histogram_freq=0, write_graph=False) ]
# 30-5-1 tanh
ranking_model.fit_generator( batch_generator(idx_pairs_train), 
                    samples_per_epoch = len(idx_pairs_train)//BATCH_SZ*BATCH_SZ,
                    nb_epoch=NB_EPOCH,
                    validation_data=batch_generator(idx_pairs_val),
                    nb_val_samples=len(idx_pairs_val)//BATCH_SZ*BATCH_SZ, 
                    callbacks = _callbacks)

Epoch 1/20
399488/399488 [==============================] - 27s - loss: 0.6998 - ranking_acc: 0.7107 - val_loss: 0.7042 - val_ranking_acc: 0.7083
Epoch 2/20
399488/399488 [==============================] - 26s - loss: 0.6990 - ranking_acc: 0.7109 - val_loss: 0.7033 - val_ranking_acc: 0.7083
Epoch 3/20
399488/399488 [==============================] - 26s - loss: 0.6983 - ranking_acc: 0.7112 - val_loss: 0.7027 - val_ranking_acc: 0.7093
Epoch 4/20
399488/399488 [==============================] - 26s - loss: 0.6976 - ranking_acc: 0.7115 - val_loss: 0.7019 - val_ranking_acc: 0.7088
Epoch 5/20
399488/399488 [==============================] - 27s - loss: 0.6969 - ranking_acc: 0.7120 - val_loss: 0.7009 - val_ranking_acc: 0.7096
Epoch 6/20
399488/399488 [==============================] - 28s - loss: 0.6960 - ranking_acc: 0.7125 - val_loss: 0.7010 - val_ranking_acc: 0.7100
Epoch 7/20
399488/399488 [==============================] - 24s - loss: 0.6952 - ranking_acc: 0.7128 - val_loss: 0.6991 - va

------------------

## below are some testing stuff

In [204]:
zip(pos_ids[0][:10], neg_ids[0][:10])

[('3429740', '3921765'),
 ('2999735', '4085271'),
 ('3526517', '4532844'),
 ('1750992', '2769307'),
 ('3503351', '3809984'),
 ('4130891', '4685986'),
 ('4471306', '2996340'),
 ('3286730', '4395018'),
 ('4659952', '28994'),
 ('4724023', '4716452')]

In [263]:
query = QUERIES[0]
pos_sample = get_query_doc_feature(query, '3429740')
neg_sample = get_query_doc_feature(query, '3921765')
pair_sample = np.array([pos_sample, neg_sample])
_idf = np.array([get_idf(wd) for wd in query])
idf_sample = np.vstack([_idf]*2)

print idf_sample.shape, pair_sample.shape

(2, 40) (2, 40, 30)


In [264]:
pos_sample[:3]

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   4.,  46.,  36.,  22.,   7.,   0.,   3.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   1.,   5.,  21.,  32.,  43.,  10.,   5.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   8.,  38.,  58.,   9.,   5.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

### test `scoring_model`

In [265]:
scoring_model.predict([idf_sample,pair_sample])

array([ 0.64522916, -0.64596307], dtype=float32)

In [242]:
a = feed_forward.predict(pos_sample)
print a

[[-0.9996202 ]
 [-0.99949074]
 [-0.99962032]
 [-0.99962097]
 [ 0.9845562 ]
 [ 0.90144622]
 [ 0.99849176]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]
 [-0.86364526]]


In [243]:
b = gating.predict(idf_sample)[0]
print b

[ 0.12651604  0.00871993  0.00948967  0.02779949  0.09881964  0.07380101
  0.65485418  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]


In [244]:
b.dot(a)

array([ 0.64522916], dtype=float32)

In [245]:
c = feed_forward.predict(neg_sample)
# print c
print b.dot(c)

[-0.64596307]


==> the scoring model works all right

### test ranking_model

In [212]:
ranking_model.predict( [idf_sample, pair_sample, np.array([neg_sample, pos_sample]) ])

array([ 0.244481, -0.244481], dtype=float32)

In [213]:
0.08474284 - -0.15973815

0.24448099

In [253]:
def predict_score(pmcid):
    query = QUERIES[0]
    _idf = np.array([get_idf(wd) for wd in query])
    _idf = np.vstack([_idf])
    _hist = get_query_doc_feature(query, pmcid).reshape(1,N,30)
    return scoring_model.predict([_idf, _hist])[0]

In [250]:
 query = QUERIES[0]
_idf = np.array([get_idf(wd) for wd in query])
_idf = np.vstack([_idf])
# pos_sample = get_query_doc_feature(query, '3429740')
scoring_model.predict( [_idf, pos_sample.reshape(1,N,30)])

array([ 0.64522916], dtype=float32)

In [251]:
predict_score('3429740')

array([ 0.64522916], dtype=float32)

### see some results

In [254]:
zip( map(predict_score, pos_ids[0][:10]), map(predict_score, neg_ids[0][:10]))

[(0.64522916, -0.64596307),
 (0.58043218, -0.80223674),
 (0.43058032, -0.96593964),
 (0.65287751, 0.45643875),
 (0.34601948, -0.64428258),
 (0.14929375, -0.085226007),
 (0.29558107, -0.85025477),
 (0.33852708, -0.032222524),
 (0.62793219, -0.96636206),
 (0.64027685, -0.76521266)]

In [255]:
def predict_score_diff( (pmcid_pos, pmcid_neg) ):
    query = QUERIES[0]
    _idf = np.array([get_idf(wd) for wd in query])
    _idf = np.vstack([_idf])
    hist_pos = get_query_doc_feature(query, pmcid_pos).reshape((1,11,30))
    hist_neg = get_query_doc_feature(query, pmcid_neg).reshape((1,11,30))
    return ranking_model.predict([_idf, hist_pos, hist_neg])[0]

### test the scoring model (metrics=AP )

In [256]:
def AP(pos_scores, neg_scores):
    Q = len(pos_scores)
    pos_tags = [1] * len(pos_scores)
    neg_tags = [0] * len(neg_scores)
    all_tagged = zip(pos_scores, pos_tags) + zip(neg_scores, neg_tags)
    ranked_list = sorted(all_tagged, reverse=True)
    print ranked_list[:20]
    ranked_tag = zip(*ranked_list)[1]
    print ranked_tag[:20]
    precision_at_i = []
    corr, total = 0.0, 0
    while corr<Q:
        if ranked_tag[total]==1: 
            corr += 1
            precision_at_i.append(corr*1.0 / (total+1) )
        total += 1
    print precision_at_i[:20]
    return np.mean(precision_at_i)

In [257]:
def AP_of_topic(q):
    query = QUERIES[q]
    _idf = np.array([get_idf(wd) for wd in query])
    _idfs = np.vstack([_idf]*len(hists_pos[q]))
    pos_scores = scoring_model.predict( [ _idfs, hists_pos[q]])
    _idfs = np.vstack([_idf]*len(hists_neg[q]))
    neg_scores = scoring_model.predict( [ _idfs, hists_neg[q]])
    print 'mean:', pos_scores.mean(), neg_scores.mean()
    print 'max:',pos_scores.max(), neg_scores.max()
    print 'min:',pos_scores.min(), neg_scores.min()
    return AP(pos_scores, neg_scores)

In [258]:
AP_of_topic(0)

mean: 0.238764 -0.517441
max: 0.805651 0.803635
min: -0.939749 -0.998747
[(0.80565107, 1), (0.8036353, 0), (0.77087092, 0), (0.74555987, 0), (0.73883641, 0), (0.72641373, 0), (0.72107649, 0), (0.71497041, 1), (0.70288318, 0), (0.69102049, 0), (0.68912357, 0), (0.67798567, 1), (0.67725968, 1), (0.67560613, 0), (0.66907352, 0), (0.66331464, 0), (0.661304, 1), (0.65667295, 0), (0.65469623, 1), (0.65364259, 0)]
(1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0)
[1.0, 0.25, 0.25, 0.3076923076923077, 0.29411764705882354, 0.3157894736842105, 0.3333333333333333, 0.32, 0.32142857142857145, 0.3333333333333333, 0.3333333333333333, 0.35294117647058826, 0.3170731707317073, 0.2978723404255319, 0.3125, 0.32, 0.3333333333333333, 0.34615384615384615, 0.35185185185185186, 0.36363636363636365]


0.35291676742918116

In [259]:
AP_of_topic(1)

mean: -0.929451 -0.874651
max: -0.833393 0.819759
min: -0.979597 -0.99962
[(0.81975895, 0), (0.77411568, 0), (0.75447631, 0), (0.74748838, 0), (0.7213372, 0), (0.71550083, 0), (0.67871034, 0), (0.63185596, 0), (0.62454158, 0), (0.61594874, 0), (0.5598042, 0), (0.55463445, 0), (0.547144, 0), (0.4956826, 0), (0.44292343, 0), (0.39915949, 0), (0.35278359, 0), (0.27660221, 0), (0.20339371, 0), (0.20259269, 0)]
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0.008928571428571428, 0.01652892561983471, 0.0234375, 0.02857142857142857, 0.03424657534246575, 0.02654867256637168, 0.03070175438596491, 0.029850746268656716, 0.03345724907063197, 0.034013605442176874, 0.03363914373088685, 0.03296703296703297, 0.0325, 0.033734939759036145, 0.033632286995515695, 0.03547671840354767, 0.03632478632478633, 0.0379746835443038, 0.0395010395010395, 0.04106776180698152]


0.034055118151864051

In [260]:
AP_of_topic(2)

mean: 0.288273 0.0659688
max: 0.736521 0.78059
min: -0.914312 -0.971865
[(0.78059, 0), (0.7555573, 0), (0.75069457, 0), (0.74577516, 0), (0.74152535, 0), (0.73652124, 1), (0.73498589, 0), (0.73440605, 0), (0.73362786, 0), (0.73230743, 1), (0.7317878, 0), (0.73043716, 0), (0.72941667, 0), (0.72841209, 0), (0.72787756, 0), (0.72695607, 0), (0.7266326, 0), (0.72596002, 0), (0.72514492, 0), (0.72453558, 0)]
(0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[0.16666666666666666, 0.2, 0.12, 0.11428571428571428, 0.1282051282051282, 0.14634146341463414, 0.16279069767441862, 0.18181818181818182, 0.19148936170212766, 0.20833333333333334, 0.21568627450980393, 0.20689655172413793, 0.20967741935483872, 0.21212121212121213, 0.1595744680851064, 0.1553398058252427, 0.1588785046728972, 0.1651376146788991, 0.17272727272727273, 0.15384615384615385]


0.15607422336388282

In [261]:
AP_of_topic(3)

mean: 0.59509 0.416526
max: 0.812807 0.886073
min: 0.289502 -0.973874
[(0.88607287, 0), (0.82710886, 0), (0.81280696, 1), (0.80342525, 0), (0.80238789, 0), (0.79907805, 0), (0.79476434, 0), (0.79390699, 0), (0.79306328, 1), (0.7904945, 0), (0.78551912, 0), (0.78282005, 0), (0.78264034, 0), (0.78159118, 1), (0.78129673, 0), (0.77883512, 0), (0.7717362, 0), (0.76929158, 0), (0.76877606, 0), (0.76352835, 0)]
(0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0)
[0.3333333333333333, 0.2222222222222222, 0.21428571428571427, 0.021164021164021163, 0.024271844660194174, 0.023166023166023165, 0.022364217252396165, 0.0196078431372549, 0.017475728155339806, 0.019083969465648856, 0.019963702359346643, 0.019077901430842606, 0.017711171662125342, 0.01629802095459837, 0.01646542261251372, 0.01737242128121607, 0.016815034619188922, 0.01637852593266606]


0.058725395427480329

In [262]:
AP_of_topic(4)

mean: 0.0427221 -0.189777
max: 0.502116 0.649829
min: -0.789292 -0.931807
[(0.64982873, 0), (0.61350065, 0), (0.61141121, 0), (0.53826547, 0), (0.53825432, 0), (0.52572823, 0), (0.51764077, 0), (0.50696397, 0), (0.50211555, 1), (0.48248726, 0), (0.48068345, 0), (0.47728133, 1), (0.47323179, 0), (0.46977085, 0), (0.46935955, 1), (0.46806246, 1), (0.46671361, 1), (0.46537223, 0), (0.4557724, 0), (0.45570201, 0)]
(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0)
[0.1111111111111111, 0.16666666666666666, 0.2, 0.25, 0.29411764705882354, 0.1935483870967742, 0.21875, 0.23529411764705882, 0.2, 0.19607843137254902, 0.2037037037037037, 0.21428571428571427, 0.20634920634920634, 0.2028985507246377, 0.20833333333333334, 0.2191780821917808, 0.22666666666666666, 0.23076923076923078, 0.22093023255813954, 0.22727272727272727]


0.14501357920381305